# Contaminated strings

From a response to 
<center>
<a target="_blank" rel="noopener noreferrer" href="http://www.youtube.com/watch?v=oHCBFrEglMA"><img alt="Video: Solving Week 83 of Perl Weekly Challenge in APL" src="https://github.com/rikedyp/JupyterNotebooks/raw/master/img/PWC83.png" width="500px"></a>
</center>

<pre>
    Hello. Small additional challenge: When there is a star in a string, it is contaminated. 'ab*' -> '***'.  
    s ← 'ab' 'c*dD' 'eEE*' '**' 'h' will give 'ab'  '****'  '****'  '**' 'h'  
    First idea : (v/s)←(⍴¨(v←'*'∊¨s)/s)⍴¨'*'  
    Or : virus←{(v/s)←(⍴¨(v←'*'∊¨s)/s←⍵)⍴¨'*'⋄s}  
    Would you have a better solution? Thanks
</pre>    

<a target="_blank" rel="noopener noreferrer" href="https://www.youtube.com/channel/UCIveI1ySN5FBe4GyiIJCDig">Schraf : Maths-info</a>

First, let's define some inputs:

In [1]:
small   ← 'ab' 'c*dD' 'eEE*' '**' 'h'
MakeArg ← {⍺←42 ⋄ ⎕RL←⍺ ⋄ (⎕A,'*')⊂⍤⌷⍤0 99⍨?∘27¨(?⍵⍴10)}
medium  ← MakeArg 1000
large   ← MakeArg 100000

I am going to rephrase Schraf's solution slightly, to my own taste:

In [2]:
Schraf ← {s←⍵ ⋄ m←⍺∘∊¨s ⋄ (m⌿s)←m⌿⍴∘⍺¨≢¨s ⋄ s}
'*' Schraf small

┌──┬────┬────┬──┬─┐
│ab│****│****│**│h│
└──┴────┴────┴──┴─┘

For some people, using a guard might feel more natural if we have to apply to each of a nested vector:

In [3]:
Guard ← {⍺∊⍵: ⍺⍴⍨≢⍵ ⋄ ⍵}   ⍝ If there is a star, ≢⍵ stars, else ⍵
'*'∘Guard¨ small

┌──┬────┬────┬──┬─┐
│ab│****│****│**│h│
└──┴────┴────┴──┴─┘

If it is possible to do so, using flat rectangular arrays can be more performant: 

In [4]:
Flat ← {m←⍺∨/⍤=s←⍵ ⋄ (m⌿s)←m⌿⍺⍴⍤0⍨+/' '≠s ⋄ s}
flatsmall ← ↑small
'*' Flat flatsmall

ab 
****
****
** 
h

In [5]:
]runtime -c "'*'Schraf small" "'*'∘Guard¨small" "'*' Flat flatsmall"

'*'Schraf small → 3.2E¯6 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 '*'∘Guard¨small → 2.2E¯6 | -33% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* '*' Flat flatsmall → 3.3E¯6 | +1% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

... but not always.  
Be aware that `]runtime` differences of <10% aren't particularly reliable. This is my assertion, although possibly someone with more knowledge can give a more accurate margin of error.

Now let's see how medium and large sized inputs compare:

In [6]:
flatmed   ← ↑ medium
flatlarge ← ↑ large

In [7]:
]runtime -c "'*'Schraf medium" "'*'∘Guard¨medium" "'*' Flat flatmed"

'*'Schraf medium → 2.1E¯4 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 '*'∘Guard¨medium → 2.3E¯4 | +10% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* '*' Flat flatmed → 1.5E¯4 | -30% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

In [8]:
]runtime -c "'*'Schraf large" "'*'∘Guard¨large" "'*' Flat flatlarge"

'*'Schraf large → 2.2E¯2 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 '*'∘Guard¨large → 2.8E¯2 | +24% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* '*' Flat flatlarge → 1.6E¯2 | -28% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

Of course there will be extra processing involved if we need a nested result for the flat version.

In [9]:
~∘' '¨↓ '*'Flat ↑small

┌──┬────┬────┬──┬─┐
│ab│****│****│**│h│
└──┴────┴────┴──┴─┘

In [10]:
]runtime -c "'*'Schraf large" "'*'∘Guard¨large" "~∘' '¨↓ '*' Flat ↑large"

'*'Schraf large → 2.2E¯2 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 '*'∘Guard¨large → 2.6E¯2 | +18% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 ~∘' '¨↓ '*' Flat ↑large → 2.2E¯2 | +2% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕